In [1]:
import pandas as pd
from tabulate import tabulate  

df_filtered = pd.read_csv('filtered_lifts_data.csv')
df_men = df_filtered[(df_filtered['Sex'] == 'M') & (df_filtered['Place'] >= 1) & (df_filtered['Place'] <= 9)]

lifts = ['Squat', 'Bench', 'Deadlift']
attempts = [f"{lift}{i}Kg" for lift in lifts for i in range(1, 4)]
results = [f"{lift}{i}Result" for lift in lifts for i in range(1, 4)]

for col in attempts:
    df_men.loc[:, col] = df_men[col].abs()

grouped = df_men.groupby(['WeightClassKg', 'Place'])
progression_relative_to_max = []

for (weight_class, place), group in grouped:
    for lift in lifts:
        max_lift = 0
        for i in range(1, 4):
            lift_col = f"{lift}{i}Kg"
            result_col = f"{lift}{i}Result"
            successful_lift = group[group[result_col] == 1][lift_col]
            if not successful_lift.empty:
                max_lift = max(max_lift, successful_lift.max())

        if max_lift > 0:
            lift1 = group[f"{lift}1Kg"]
            lift2 = group[f"{lift}2Kg"]
            lift3 = group[f"{lift}3Kg"]
            
            progression_1_to_2 = round(((lift2.mean() - lift1.mean()) / max_lift * 100), 2) if lift1.mean() > 0 else 0
            progression_2_to_3 = round(((lift3.mean() - lift2.mean()) / max_lift * 100), 2) if lift2.mean() > 0 else 0
            
            progression_relative_to_max.append({
                "Weight Class": weight_class,
                "Place": place,
                "Lift": lift,
                "1st to 2nd Attempt (%)": f"{progression_1_to_2:.2f}%",
                "2nd to 3rd Attempt (%)": f"{progression_2_to_3:.2f}%"
            })
        else:
            progression_relative_to_max.append({
                "Weight Class": weight_class,
                "Place": place,
                "Lift": lift,
                "1st to 2nd Attempt (%)": "0.00%",
                "2nd to 3rd Attempt (%)": "0.00%"
            })

print(tabulate(progression_relative_to_max, headers="keys", tablefmt="grid", showindex=False))


+----------------+---------+----------+--------------------------+--------------------------+
|   Weight Class |   Place | Lift     | 1st to 2nd Attempt (%)   | 2nd to 3rd Attempt (%)   |
+================+=========+==========+==========================+==========================+
|           60   |       1 | Squat    | 2.95%                    | 1.32%                    |
+----------------+---------+----------+--------------------------+--------------------------+
|           60   |       1 | Bench    | 3.81%                    | 1.76%                    |
+----------------+---------+----------+--------------------------+--------------------------+
|           60   |       1 | Deadlift | 3.79%                    | 2.46%                    |
+----------------+---------+----------+--------------------------+--------------------------+
|           60   |       2 | Squat    | 5.18%                    | 3.82%                    |
+----------------+---------+----------+---------------------

In [2]:
import pandas as pd
from tabulate import tabulate

df_filtered = pd.read_csv('filtered_lifts_data.csv')
df_men = df_filtered[(df_filtered['Sex'] == 'M') & (df_filtered['Place'] >= 1) & (df_filtered['Place'] <= 9)]

lifts = ['Squat', 'Bench', 'Deadlift']
results = [f"{lift}{i}Result" for lift in lifts for i in range(1, 4)]

success_rates_by_weightclass = {}
grouped = df_men.groupby('WeightClassKg')

for weight_class, group in grouped:
    success_rates_by_weightclass[weight_class] = {}
    
    for lift in lifts:
        success_rates_by_weightclass[weight_class][lift] = {}
        
        for place in range(1, 10):
            place_df = group[group['Place'] == place]
            total_lifters = len(place_df)
            successful_attempts = sum([(place_df[f"{lift}{i}Result"] == 1).sum() for i in range(1, 4)])
            success_rate = (successful_attempts / (3 * total_lifters) * 100) if total_lifters > 0 else 0
            success_rates_by_weightclass[weight_class][lift][place] = round(success_rate, 2)

table_data = []

for weight_class, lifts_data in success_rates_by_weightclass.items():
    for lift, places_data in lifts_data.items():
        row = {"Weight Class": weight_class, "Lift": lift}
        row.update({f"Place {place}": f"{places_data[place]:.2f}%" for place in range(1, 10)})
        table_data.append(row)

table_df = pd.DataFrame(table_data)
print(tabulate(table_df, headers="keys", tablefmt="grid", showindex=False))


+----------------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|   Weight Class | Lift     | Place 1   | Place 2   | Place 3   | Place 4   | Place 5   | Place 6   | Place 7   | Place 8   | Place 9   |
+================+==========+===========+===========+===========+===========+===========+===========+===========+===========+===========+
|           60   | Squat    | 77.88%    | 74.07%    | 63.64%    | 66.67%    | 40.00%    | 77.78%    | 58.33%    | 66.67%    | 66.67%    |
+----------------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|           60   | Bench    | 73.40%    | 64.81%    | 60.61%    | 69.44%    | 73.33%    | 72.22%    | 75.00%    | 66.67%    | 50.00%    |
+----------------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|           60   | Deadlift | 81.7

In [3]:
import pandas as pd
from tabulate import tabulate

df_filtered = pd.read_csv('filtered_lifts_data.csv')
df_men = df_filtered[(df_filtered['Sex'] == 'M') & (df_filtered['Place'] >= 1) & (df_filtered['Place'] <= 9)]

lifts = ['Squat', 'Bench', 'Deadlift']
results = [f"{lift}{i}Result" for lift in lifts for i in range(1, 4)]

for col in [f"{lift}{i}Kg" for lift in lifts for i in range(1, 4)]:
    df_men.loc[:, col] = df_men[col].abs()

grouped_by_place = df_men.groupby(['Place'])
progression_and_success = []

for place, group in grouped_by_place:
    for lift in lifts:
        lift1_mean = group[f"{lift}1Kg"].mean()
        lift2_mean = group[f"{lift}2Kg"].mean()
        lift3_mean = group[f"{lift}3Kg"].mean()

        valid_lift3 = group[f"{lift}3Kg"][group[f"{lift}3Result"] == 1].mean()
        valid_lift2 = group[f"{lift}2Kg"][group[f"{lift}2Result"] == 1].mean()

        if not pd.isna(valid_lift3) and valid_lift3 > 0:
            first_attempt_percent = round((lift1_mean / valid_lift3) * 100, 2)
        elif not pd.isna(valid_lift2) and valid_lift2 > 0:
            first_attempt_percent = round((lift1_mean / valid_lift2) * 100, 2)
        else:
            first_attempt_percent = 0

        if lift1_mean > 0:
            progression_1_to_2 = round(((lift2_mean - lift1_mean) / lift1_mean) * 100, 2)
        else:
            progression_1_to_2 = 0

        if lift2_mean > 0:
            progression_2_to_3 = round(((lift3_mean - lift2_mean) / lift2_mean) * 100, 2)
        else:
            progression_2_to_3 = 0

        total_lifters = len(group)
        successful_1 = (group[f"{lift}1Result"] == 1).sum()
        success_rate_1 = round((successful_1 / total_lifters) * 100, 2) if total_lifters > 0 else 0

        successful_2 = (group[f"{lift}2Result"] == 1)
        total_successful_1 = (group[f"{lift}1Result"] == 1).sum()
        success_rate_1_to_2 = (
            (successful_2 & successful_1).sum() / total_successful_1 * 100
        ) if total_successful_1 > 0 else 0

        successful_3 = (group[f"{lift}3Result"] == 1)
        total_successful_2 = successful_2.sum()
        success_rate_2_to_3 = (
            (successful_3 & successful_2).sum() / total_successful_2 * 100
        ) if total_successful_2 > 0 else 0

        progression_and_success.append({
            "Place": place,
            "Lift": lift,
            "1st Attempt (% of Final)": first_attempt_percent,
            "Success Rate 1st Attempt (%)": success_rate_1,
            "Progression 1st to 2nd (%)": progression_1_to_2,
            "Success Rate 1st to 2nd (%)": round(success_rate_1_to_2, 2),
            "Progression 2nd to 3rd (%)": progression_2_to_3,
            "Success Rate 2nd to 3rd (%)": round(success_rate_2_to_3, 2)
        })

progression_and_success_df = pd.DataFrame(progression_and_success)
csv_filename = "progression_and_success_rates.csv"
progression_and_success_df.to_csv(csv_filename, index=False)
print(tabulate(progression_and_success_df, headers="keys", tablefmt="grid", showindex=False))
print(f"\nThe table has been saved to '{csv_filename}'.")


+---------+----------+----------------------------+--------------------------------+------------------------------+-------------------------------+------------------------------+-------------------------------+
| Place   | Lift     |   1st Attempt (% of Final) |   Success Rate 1st Attempt (%) |   Progression 1st to 2nd (%) |   Success Rate 1st to 2nd (%) |   Progression 2nd to 3rd (%) |   Success Rate 2nd to 3rd (%) |
+=========+==========+============================+================================+==============================+===============================+==============================+===============================+
| (1,)    | Squat    |                      92.42 |                          88.18 |                         6.07 |                         93.48 |                         3.83 |                         65.13 |
+---------+----------+----------------------------+--------------------------------+------------------------------+-------------------------------+---------

In [4]:
import random
import ast
import pandas as pd
import numpy as np
from collections import defaultdict

# RL Agent class
class ReinforcementLearner:
    def __init__(self):
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.actions = ["conservative", "balanced", "aggressive"]
        self.alpha = 0.1
        self.gamma = 0.9
        self.epsilon = 0.2

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.choice(self.actions)
        return max(self.actions, key=lambda action: self.q_table[state][action])

    def update_q_value(self, state, action, reward, next_state):
        best_next_action = max(self.actions, key=lambda a: self.q_table[next_state][a])
        td_target = reward + self.gamma * self.q_table[next_state][best_next_action]
        td_error = td_target - self.q_table[state][action]
        self.q_table[state][action] += self.alpha * td_error




        
    

In [5]:
def simulate_competition(progression_success_data, rl_agent=None):
    def simulate_attempt(weight, success_prob):
        return random.random() < success_prob

    def simulate_lift_attempts(agent, lift_type, place_strategies=None, rl_action=None):
        max_lift = agent[f"Max{lift_type}"]

        # Extract progression ranges from the CSV
        first_attempt_factor = place_strategies.get("1st Attempt (% of Final)", 92) / 100
        min_prog_1_to_2 = (place_strategies.get("Progression 1st to 2nd (%)", 5.0) - 1.0) / 100 + 1  # Range min
        max_prog_1_to_2 = (place_strategies.get("Progression 1st to 2nd (%)", 7.0) - 1.0) / 100 + 1  # Range max
        min_prog_2_to_3 = (place_strategies.get("Progression 2nd to 3rd (%)", 5.0) - 1.0) / 100 + 1
        max_prog_2_to_3 = (place_strategies.get("Progression 2nd to 3rd (%)", 8.0) - 1.0) / 100 + 1

       
        if rl_action:
            progression_1_to_2 = random.uniform(min_prog_1_to_2, max_prog_1_to_2)
            progression_2_to_3 = random.uniform(min_prog_2_to_3, max_prog_2_to_3)
        else:
            progression_1_to_2 = (place_strategies.get("Progression 1st to 2nd (%)", 6.07) / 100) + 1
            progression_2_to_3 = (place_strategies.get("Progression 2nd to 3rd (%)", 6.07) / 100) + 1

        progression_factors = [first_attempt_factor, progression_1_to_2, progression_2_to_3]

        # Initialize attempt weights
        attempt_weights = []
        current_weight = int(max_lift * progression_factors[0])  # First attempt
        attempt_weights.append(current_weight)

        # Incrementally increase weights for subsequent attempts
        for progression_factor in progression_factors[1:]:
            next_weight = int(current_weight * progression_factor)
            attempt_weights.append(next_weight)
            current_weight = next_weight  # Update weight for the next progression

        # Determine success rates based on CSV file values
        success_rates = [
            place_strategies.get("Success Rate 1st Attempt (%)", 95) / 100,
            np.random.normal(place_strategies.get("Success Rate 1st to 2nd (%)", 90), 1) / 100,
            np.random.normal(place_strategies.get("Success Rate 2nd to 3rd (%)", 85), 1) / 100,
        ]

        # Simulate attempts
        results = []
        for i, (weight, success_prob) in enumerate(zip(attempt_weights, success_rates)):
            success = simulate_attempt(weight, success_prob)
            if not success and i > 0:  # RL failure adjustment
                next_conservative_factor = 1.02  # Define conservative progression factor after failure
                weight = int(weight * next_conservative_factor)  # Increase weight but conservatively
            results.append({"Weight": weight, "Success": success})

        return results

    # Randomize agents' totals and places
    def randomize_agents():
        agents = []
        names = [f"Lifter {i}" for i in range(1, 10)]  # 9 lifters
        places = list(range(1, 10))
        random.shuffle(places)  # Shuffle places

        # Define lifter distributions
        mean_squat, std_squat = 250, 7.5
        mean_bench, std_bench = 150, 5
        mean_deadlift, std_deadlift = 300, 10

        for name, place in zip(names, places):
            agents.append({
                "Name": name,
                "Place": place,  # Assign random Place
                "MaxSquat": max(0, int(np.random.normal(mean_squat, std_squat))),
                "MaxBench": max(0, int(np.random.normal(mean_bench, std_bench))),
                "MaxDeadlift": max(0, int(np.random.normal(mean_deadlift, std_deadlift)))
            })
        return agents

    # Ensure "Place" column is properly formatted
    progression_success_data["Place"] = progression_success_data["Place"].apply(
        lambda x: x[0] if isinstance(x, tuple) else x
    )

    # Randomize agent details
    agents = randomize_agents()
    rl_lifter = agents[0]  # Assume the first lifter is the RL lifter
    rl_lifter["Name"] = "RL Lifter"  # Name the RL lifter explicitly

    results_data = []
    for agent in agents:
        pre_total = sum(agent[f"Max{lift}"] for lift in ["Squat", "Bench", "Deadlift"])
        result_record = {
            "Name": agent["Name"],
            "PreTotal": pre_total,
            "PostTotal": 0,
            "Type": "RL Agent" if agent["Name"] == "RL Lifter" else "Standard Lifter"
        }

        place = agent["Place"]
        result_record["Place Strategy Value"] = place

        # Extract Place-based strategy or fallback
        if place in progression_success_data["Place"].values:
            place_strategies = progression_success_data[progression_success_data["Place"] == place].to_dict(orient="records")[0]
        else:
            # Default/fallback strategy
            place_strategies = {
                "1st Attempt (% of Final)": 92,
                "Progression 1st to 2nd (%)": 6.07,
                "Progression 2nd to 3rd (%)": 6.07,
                "Success Rate 1st Attempt (%)": 95,
                "Success Rate 1st to 2nd (%)": 90,
                "Success Rate 2nd to 3rd (%)": 85
            }

        # Simulate each lift
        for lift in ["Squat", "Bench", "Deadlift"]:
            if agent["Name"] == "RL Lifter" and rl_agent:
                state = "start"

                # Epsilon-greedy exploration
                epsilon = 0.5
                if random.random() < epsilon:  # Explore
                    action = random.choice(["conservative", "balanced", "aggressive"])
                else:  # Exploit
                    action = rl_agent.choose_action(state)

                lift_results = simulate_lift_attempts(agent, lift, place_strategies, rl_action=action)

                # Reward riskier attempts
                max_attempted = max((r["Weight"] for r in lift_results), default=0)
                reward = max((r["Weight"] for r in lift_results if r["Success"]), default=0)
                reward += 0.2 * max_attempted  # Bonus for attempting heavier weights

                next_state = f"{lift}_completed"
                rl_agent.update_q_value(state, action, reward, next_state)
                state = next_state
            else:
                # Non-RL lifters follow Place-based strategy
                lift_results = simulate_lift_attempts(agent, lift, place_strategies)

            for i, attempt in enumerate(lift_results, start=1):
                result_record[f"{lift}_Attempt_{i}_Weight"] = attempt["Weight"]
                result_record[f"{lift}_Attempt_{i}_Success"] = attempt["Success"]
            best_successful = max((r["Weight"] for r in lift_results if r["Success"]), default=0)
            result_record["PostTotal"] += best_successful

        results_data.append(result_record)

    results_df = pd.DataFrame(results_data)
    results_df["Rank"] = results_df["PostTotal"].rank(ascending=False, method="dense").astype(int)
    results_df = results_df.sort_values(by="Rank")

    # Save results to a CSV file
    csv_filename = "competition_results_with_place_and_rl4.csv"
    results_df.to_csv(csv_filename, index=False)
    print(f"Simulation results saved to '{csv_filename}'.")
    return results_df


# Example usage:
progression_success_data = pd.read_csv("progression_and_success_rates.csv")
progression_success_data["Place"] = progression_success_data["Place"].apply(ast.literal_eval)
rl_agent = ReinforcementLearner()
results = simulate_competition(progression_success_data, rl_agent=rl_agent)

Simulation results saved to 'competition_results_with_place_and_rl4.csv'.
